In [104]:
import pandas as pd
import string
import requests
from bs4 import BeautifulSoup
import numpy as np
from typing import Final


# DEFINES
RES_PATH: Final = r"./resources/software_developer_united_states_1971_20191023_1.csv"

LANG_LIST: Final = [
    ["java"],
    ["python"],
    ["c"],
    ["kotlin"],
    ["swift"],
    ["rust"],
    ["ruby"],
    ["scala"],
    ["julia"],
    ["lua"],
]


In [105]:
# Load Data

rows = pd.read_csv(RES_PATH)
print(rows.columns)
rows.head(1)


Index(['crawl_timestamp', 'url', 'job_title', 'category', 'company_name',
       'city', 'state', 'country', 'inferred_city', 'inferred_state',
       'inferred_country', 'post_date', 'job_description', 'job_type',
       'salary_offered', 'job_board', 'geo', 'cursor', 'contact_email',
       'contact_phone_number', 'uniq_id'],
      dtype='object')


,crawl_timestamp,url,job_title,category,company_name,city,state,country,inferred_city,inferred_state,...,post_date,job_description,job_type,salary_offered,job_board,geo,cursor,contact_email,contact_phone_number,uniq_id
0,2019-02-06 05:35:27 +0000,https://www.careerbuilder.com/job/J3W7NK6NM05P...,Sr. Software Developer,architecture and engineering,Aerotek,Raleigh,NC,Usa,Raleigh,North carolina,...,2019-02-05,The chosen Sr. Software Developer will be part...,Full Time,NaN,careerbuilder,usa,1549432819204600,NaN,2604598088,f986480a8dabeea299b732335c3ffc67


In [106]:
# Transform Data
def clean_text(s: pd.Series):
    return s.apply(
        lambda s: s.lower()
        .translate(str.maketrans("", "", string.punctuation + "\xa0"))
        .translate(str.maketrans(string.whitespace, " " * len(string.whitespace)))
    )


def tokenize(s: pd.Series):
    return s.apply(lambda s: [x.strip() for x in s.split()])


def transformation_pipe(df: pd.DataFrame):
    df["job_description"] = clean_text(df["job_description"])
    df["job_description"] = tokenize(df["job_description"])
    return df


df = transformation_pipe(rows.copy())
df["job_description"].head(5)


0    [the, chosen, sr, software, developer, will, b...
1    [position, c, lead, software, developer, locat...
2    [senior, software, developer, hoboken, nj, sta...
3    [our, client, a, multinational, publishing, an...
4    [position, c, lead, software, developer, locat...
Name: job_description, dtype: object

In [107]:
# Extracting a list of database names from ranking site


def parse_db(head_size: int):
    res = requests.get(r"https://db-engines.com/en/ranking").content
    soup = BeautifulSoup(res, features="html.parser")
    return [
        # Get top-level text and transform
        element.find(string=True, recursive=False).strip().lower().split(" ")
        # Select using a selector
        for element in soup.select(r"table.dbi th[class='pad-l']>a")
    ][:head_size]


dbs: list[list[str]] = parse_db(20)
db_names = [" ".join(d) for d in dbs]

all_terms = LANG_LIST + dbs

dbs


[['oracle'],
 ['mysql'],
 ['microsoft', 'sql', 'server'],
 ['postgresql'],
 ['mongodb'],
 ['redis'],
 ['elasticsearch'],
 ['ibm', 'db2'],
 ['sqlite'],
 ['microsoft', 'access'],
 ['snowflake'],
 ['cassandra'],
 ['mariadb'],
 ['splunk'],
 ['microsoft', 'azure', 'sql', 'database'],
 ['amazon', 'dynamodb'],
 ['databricks'],
 ['hive'],
 ['google', 'bigquery'],
 ['teradata']]

## Constructing Document-Term matrix


In [108]:
query_map = pd.DataFrame(
    df["job_description"]
    .apply(lambda s: [1 if np.all([d in s for d in db]) else 0 for db in all_terms])
    .values.tolist(),
    columns=[" ".join(d) for d in all_terms],
)
query_map


,java,python,c,kotlin,swift,rust,ruby,scala,julia,lua,...,snowflake,cassandra,mariadb,splunk,microsoft azure sql database,amazon dynamodb,databricks,hive,google bigquery,teradata
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9998,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Q1: What DB should I learn after java?  (10%)
### Approach
1. Using a conditional, we can select one that only has java matched.
2. Select only from columns that are from our database list (dbs).
3. Then sum them up (Count occurances).
4. Divide by all mentions of Java

Result shows that Oracle is most often mentioned alongside Java, accouting for 24.2% of all jobs mentioning Java
This could indicate that Oracle is the most demanded alongside Java.

In [110]:
occurances_java_relate_db = query_map[query_map["java"] == 1][db_names].sum()
occurances_java_relate_db_percentage = occurances_java_relate_db.divide(query_map["java"].sum())
occurances_java_relate_db_percentage.sort_values(ascending=False)



oracle                          0.241935
mysql                           0.100335
microsoft sql server            0.056556
postgresql                      0.054462
cassandra                       0.047340
mongodb                         0.044617
elasticsearch                   0.026183
microsoft access                0.018643
splunk                          0.015710
hive                            0.011730
redis                           0.009007
amazon dynamodb                 0.009007
ibm db2                         0.007750
microsoft azure sql database    0.003770
teradata                        0.003351
sqlite                          0.002304
mariadb                         0.001047
google bigquery                 0.000628
snowflake                       0.000628
databricks                      0.000000
dtype: float64


#### Limiting sample space to all jobs mentioning Java and any kind of database.
Result shows that Oracle is mentioned in more than half of any Java position that mentions any database. (Accouting for 60.63%)
This could be inferred as Oracle being the most demanded in Java position requiring uses of a database.

In [111]:
count_java_mentions_db = len(query_map[query_map["java"] & query_map[db_names].any(axis=1)])
occurances_java_relate_db_percentage_relative_to_db_mentions = occurances_java_relate_db.divide(count_java_mentions_db)
occurances_java_relate_db_percentage_relative_to_db_mentions.sort_values(ascending=False)


oracle                          0.606299
mysql                           0.251444
microsoft sql server            0.141732
postgresql                      0.136483
cassandra                       0.118635
mongodb                         0.111811
elasticsearch                   0.065617
microsoft access                0.046719
splunk                          0.039370
hive                            0.029396
redis                           0.022572
amazon dynamodb                 0.022572
ibm db2                         0.019423
microsoft azure sql database    0.009449
teradata                        0.008399
sqlite                          0.005774
mariadb                         0.002625
google bigquery                 0.001575
snowflake                       0.001575
databricks                      0.000000
dtype: float64

## Q2: Which DB is in demand alongside oracle? (10%)
### Approach
1. Using a conditional, we can select one that only has oracle matched.
2. Select only from columns that are from our database list (dbs) without oracle (would be all of them anyway).
3. Then sum them up (Count occurances).
4. Divide by all mentions of Oracle

Result shows that MySQL is most often mentioned alongside Oracle, accouting for 22.24% of any databases mentioned alongside Oracle.
This could indicate that MySQL is the most demanded alongside Oracle.

In [112]:
occurances_oracle_relation = query_map[query_map["oracle"] == 1][[db for db in db_names if db != "oracle"]].sum()
occurances_oracle_relation_percentage = occurances_oracle_relation.divide(query_map["oracle"].sum())
occurances_oracle_relation_percentage.sort_values(ascending=False)


mysql                           0.222416
microsoft sql server            0.124855
mongodb                         0.083043
postgresql                      0.073751
cassandra                       0.051103
splunk                          0.042393
microsoft access                0.034843
elasticsearch                   0.022067
hive                            0.018583
ibm db2                         0.015679
sqlite                          0.014518
teradata                        0.012776
amazon dynamodb                 0.008130
redis                           0.006969
microsoft azure sql database    0.005226
mariadb                         0.002904
snowflake                       0.002904
google bigquery                 0.001742
databricks                      0.001161
dtype: float64

## Q3: What programming language is in demand alongside python? (10%)
### Approach
1. Using a conditional, we can select one that only has Python matched.
2. Select only from columns that are from our programming languages list (LANG_LIST) without Python (which would be all of them anyway).
3. Then sum them up (Count occurances).
4. Divide by all mentions of Python

Result shows that Java and C/C++/C# are most often mentioned alongside Python, accouting for 64.18% and 57.17% respectively, of all positions mentioning Python.
This could indicate that Java and C/C++/C# are in demand alongside Python.

In [113]:
occurances_python_relation = query_map[query_map["python"] == 1][
    [
        lang
        for lang in [" ".join(language) for language in LANG_LIST]
        if lang != "python"
    ]
].sum()
occurances_python_relation_percentage = occurances_python_relation.divide(
    query_map["python"].sum()
)
occurances_python_relation_percentage.sort_values(ascending=False)


java      0.641814
c         0.571788
ruby      0.102267
scala     0.046851
swift     0.030227
lua       0.006045
kotlin    0.005038
rust      0.003023
julia     0.000504
dtype: float64

## Q4: Create one question beginning with "WHY" that can be answered using the data we've analyzed during this class. Then, provide your solution to the question. (70%)

### Question
Why Java is the best option when you want to get employed?

### Versatility - Wideness

In [114]:
terms_category = query_map.join(df["category"])

cat_occurance = terms_category.groupby(pd.Grouper(key="category")).apply(lambda x: x.sum()/len(x))
cat_count = terms_category.groupby(pd.Grouper(key="category")).apply(lambda x: len(x))
cat_count.name = "cat_count"

freq_map = cat_occurance.join(cat_count)


freq_map[freq_map["cat_count"] > 50]


,java,python,c,kotlin,swift,rust,ruby,scala,julia,lua,...,cassandra,mariadb,splunk,microsoft azure sql database,amazon dynamodb,databricks,hive,google bigquery,teradata,cat_count
category,,,,,,,,,,,,,,,,,,,,,
architecture and engineering,0.334601,0.123574,0.380228,0.009506,0.015209,0.000000,0.047529,0.017110,0.0,0.001901,...,0.015209,0.001901,0.007605,0.007605,0.007605,0.0,0.009506,0.000000,0.000000,526
Computer/Internet,0.482759,0.196329,0.472191,0.006674,0.018910,0.000000,0.041713,0.015017,0.0,0.001112,...,0.020578,0.002781,0.007786,0.013348,0.002781,0.0,0.011123,0.001669,0.008343,1798
arts design and media,0.296950,0.123596,0.399679,0.001605,0.022472,0.004815,0.044944,0.008026,0.0,0.003210,...,0.011236,0.000000,0.003210,0.003210,0.000000,0.0,0.009631,0.000000,0.003210,623
business and financial operations,0.230769,0.076923,0.512821,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.025641,0.000000,0.012821,0.000000,0.000000,0.0,0.000000,0.000000,0.038462,78
internet,0.222222,0.314815,0.666667,0.000000,0.000000,0.000000,0.111111,0.129630,0.0,0.074074,...,0.018519,0.000000,0.000000,0.000000,0.111111,0.0,0.000000,0.000000,0.000000,54
Sales,0.558824,0.362745,0.637255,0.009804,0.019608,0.000000,0.029412,0.029412,0.0,0.000000,...,0.000000,0.000000,0.009804,0.000000,0.009804,0.0,0.000000,0.000000,0.000000,102
Engineering/Architecture,0.309091,0.418182,0.709091,0.000000,0.000000,0.000000,0.109091,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,55
Manufacturing/Mechanical,0.423077,0.192308,0.480769,0.000000,0.057692,0.000000,0.019231,0.000000,0.0,0.000000,...,0.019231,0.019231,0.057692,0.019231,0.000000,0.0,0.000000,0.000000,0.000000,52
food services,0.270492,0.188525,0.491803,0.024590,0.032787,0.000000,0.008197,0.057377,0.0,0.016393,...,0.016393,0.000000,0.000000,0.024590,0.057377,0.0,0.008197,0.000000,0.000000,122


From the figure above (Category with 50 or more listings are chosen for consistency):
1. Java are mentioned in more than 20% of listings for most of the categories.
2. Even more on category with larger sample size (Computer/Internet: 48.28%, Arts Design and Media: 29.70%, Computer-or-internet: 66.29%)

This shows that Java is used across multiple fields, so even with people with different backgrounds, Java is still preferable.

### Frequency - Tallness


In [115]:
query_map["java"].sum()/query_map.shape[0]


0.4774

47.74% for all job listings mentions Java

In [116]:
query_map[query_map[db_names].any(axis=1)]["java"].sum()/query_map["java"].sum()


0.3990364474235442

Only 39.90% of Java job listings mentions name of a database

### Conclusion
1. Java is widely used in multiple industries, giving you a good chance in independant of your background.
2. Java is mentioned in almost half (47.74%) of all listings.
3. Only 39.90% of Java listings mentions a database, meaning if you don't want to study about databases, there are still the other 60.10% that will consider you.

#### Disclaimer
These do not account for other prerequisites and conditions nor how difficult Java is to learn as it is subjective.